# 1. Exploring and checking

In [10]:
import json
full_coco = json.load(open('/home/jess/ct_classifier_wd/data/intermediate/cls_coco.json'))

In [11]:
full_coco['images'][0]

{'file_name': 'Red Kangaroo/PCAM01_2021-05-24_17-58-33_M_1_3_7.JPG',
 'height': 254,
 'width': 349,
 'id': 'PCAM01_2021-05-24_17-58-33_M_1_3'}

In [12]:
full_coco['annotations'][0]

{'id': '8697519e-3c93-11ee-bfaa-22169906f295',
 'image_id': 'PCAM01_2021-05-24_17-58-33_M_1_3',
 'category_id': 0,
 'iscrowd': 0,
 'segmentation': [],
 'area': 88646}

In [13]:
full_coco['categories']

[{'id': 0, 'name': 'Red Kangaroo'},
 {'id': 1, 'name': 'Kangaroo'},
 {'id': 2, 'name': 'Dingo'},
 {'id': 3, 'name': 'Rabbit'},
 {'id': 4, 'name': 'Cat'},
 {'id': 5, 'name': 'Emu'},
 {'id': 6, 'name': 'Bird'},
 {'id': 7, 'name': 'Pig'},
 {'id': 8, 'name': 'Euro'},
 {'id': 9, 'name': 'Fox'},
 {'id': 10, 'name': 'Echidna'},
 {'id': 11, 'name': 'Western Grey Kangaroo'},
 {'id': 12, 'name': 'Small mammal'},
 {'id': 13, 'name': 'Other'},
 {'id': 14, 'name': 'Goat'}]

In [14]:
print(len(full_coco['images']))
print(len(full_coco['annotations']))

28673
28673


In [15]:
cat_id = []
for annotation in full_coco['annotations']:
    cat_id.append(annotation['category_id'])

In [26]:
from collections import Counter
counts = Counter(cat_id)

In [51]:
splits = []
i = 0

for item in counts.values():
    train = int(item*0.70)
    val = int(item*0.15)
    test = item-val-train
    splits.append({'category_id':i,
                   'train':train,
                   'val':val,
                   'test':test})
    i=i+1

In [52]:
splits

[{'category_id': 0, 'train': 12686, 'val': 2718, 'test': 2720},
 {'category_id': 1, 'train': 1656, 'val': 354, 'test': 356},
 {'category_id': 2, 'train': 129, 'val': 27, 'test': 29},
 {'category_id': 3, 'train': 1170, 'val': 250, 'test': 252},
 {'category_id': 4, 'train': 2099, 'val': 449, 'test': 451},
 {'category_id': 5, 'train': 417, 'val': 89, 'test': 90},
 {'category_id': 6, 'train': 908, 'val': 194, 'test': 196},
 {'category_id': 7, 'train': 46, 'val': 9, 'test': 11},
 {'category_id': 8, 'train': 51, 'val': 11, 'test': 12},
 {'category_id': 9, 'train': 28, 'val': 6, 'test': 7},
 {'category_id': 10, 'train': 16, 'val': 3, 'test': 5},
 {'category_id': 11, 'train': 206, 'val': 44, 'test': 45},
 {'category_id': 12, 'train': 536, 'val': 115, 'test': 116},
 {'category_id': 13, 'train': 25, 'val': 5, 'test': 6},
 {'category_id': 14, 'train': 91, 'val': 19, 'test': 20}]

# 2. Split

In [53]:
train_path = '/home/jess/ct_classifier_wd/data/processed/cls_train_coco.json'
val_path = '/home/jess/ct_classifier_wd/data/processed/cls_val_coco.json'
test_path = '/home/jess/ct_classifier_wd/data/processed/cls_test_coco.json'

In [54]:
# Initialize dictionaries to store data
train_coco = {'images': [], 'annotations': [], 'categories': full_coco['categories']}
val_coco = {'images': [], 'annotations': [], 'categories': full_coco['categories']}
test_coco = {'images': [], 'annotations': [], 'categories': full_coco['categories']}

# Loop through splits
for split in splits:
    category_id = split['category_id']
    train_count = split['train']
    val_count = split['val']
    test_count = split['test']

    # Filter annotations for the current category based on the counts
    category_annotations = [annotation for annotation in full_coco['annotations'] if annotation['category_id'] == category_id]

    # Split the annotations into train, val, and test
    train_coco['annotations'].extend(category_annotations[:train_count])
    val_coco['annotations'].extend(category_annotations[train_count:train_count + val_count])
    test_coco['annotations'].extend(category_annotations[train_count + val_count:train_count + val_count + test_count])

# Update image IDs for each split based on the selected annotations
train_image_ids = set(annotation['image_id'] for annotation in train_coco['annotations'])
val_image_ids = set(annotation['image_id'] for annotation in val_coco['annotations'])
test_image_ids = set(annotation['image_id'] for annotation in test_coco['annotations'])

train_coco['images'] = [image for image in full_coco['images'] if image['id'] in train_image_ids]
val_coco['images'] = [image for image in full_coco['images'] if image['id'] in val_image_ids]
test_coco['images'] = [image for image in full_coco['images'] if image['id'] in test_image_ids]

# Now, train_coco, val_coco, and test_coco contain the data based on the splits


In [61]:
with open(train_path, 'w') as file:
    json.dump(train_coco, file, indent=4)
with open(val_path, 'w') as file:
    json.dump(val_coco, file, indent=4)
with open(test_path, 'w') as file:
    json.dump(test_coco, file, indent=4)